# GitHub Actions Workflow Summary Generation

This notebook demonstrates how to generate a unified CI/CD summary for notebook workflows using bash scripting and environment variables. It addresses common issues with JSON parsing and provides robust error handling for GitHub Actions step summaries.

## Problem Analysis

The error you encountered (`Process completed with exit code 5`) is likely caused by:
1. Invalid JSON format in the `NOTEBOOKS` variable
2. `jq` failing to parse malformed JSON
3. Bash's `-e` flag causing script termination on any error

Let's walk through the proper way to handle this scenario.

## 1. Write Unified Notebook CI/CD Summary Header

First, let's create the header and timestamp for our workflow summary. This should be done safely to handle any potential issues.

In [ ]:
#!/bin/bash

# Create a mock GITHUB_STEP_SUMMARY file for demonstration
GITHUB_STEP_SUMMARY="workflow_summary.md"

# Clear any existing summary
> "$GITHUB_STEP_SUMMARY"

# Write the header with proper error handling
{
  echo "## 📊 Unified Notebook CI/CD Summary"
  echo "*Generated: $(date -u +'%Y-%m-%d %H:%M:%S UTC')*"
  echo ""
} >> "$GITHUB_STEP_SUMMARY"

echo "✅ Header written successfully"
cat "$GITHUB_STEP_SUMMARY"

## 2. Display Configuration Section

Now let's add the configuration details with proper conditional logic for optional fields.

In [ ]:
#!/bin/bash

# Set example configuration variables
EXECUTION_MODE="pr"
TRIGGER_EVENT="all"
PYTHON_VERSION="3.11"
CONDA_ENVIRONMENT=""  # Empty for demonstration
CUSTOM_REQUIREMENTS=""  # Empty for demonstration
SINGLE_NOTEBOOK=""  # Empty for demonstration

# Add configuration section
{
  echo "### 🔧 Configuration"
  echo "- **Execution Mode**: $EXECUTION_MODE"
  echo "- **Trigger Event**: $TRIGGER_EVENT"
  echo "- **Python Version**: $PYTHON_VERSION"
  
  # Only add optional fields if they have values
  if [ -n "$CONDA_ENVIRONMENT" ]; then
    echo "- **Custom Conda Environment**: $CONDA_ENVIRONMENT"
  fi
  
  if [ -n "$CUSTOM_REQUIREMENTS" ]; then
    echo "- **Custom Requirements**: $CUSTOM_REQUIREMENTS"
  fi
  
  if [ -n "$SINGLE_NOTEBOOK" ]; then
    echo "- **Single Notebook Target**: \`$SINGLE_NOTEBOOK\`"
  fi
  
  echo ""
} >> "$GITHUB_STEP_SUMMARY"

echo "✅ Configuration section added"
cat "$GITHUB_STEP_SUMMARY"

## 3. Display Job Results Table

Create a markdown table showing the status of each workflow job.

In [ ]:
#!/bin/bash

# Set job status variables
SETUP_RESULT="success"
PROCESS_RESULT="success"
BUILD_RESULT="skipped"
DEPRECATION_RESULT="skipped"

# Add job results table
{
  echo "### 📊 Job Results"
  echo "| Job | Status | Duration |"
  echo "|-----|--------|----------|"
  echo "| Matrix Setup | $SETUP_RESULT | - |"
  
  # Handle special case for merge mode
  if [ "$EXECUTION_MODE" != "merge" ]; then
    echo "| Notebook Processing | $PROCESS_RESULT | - |"
  else
    echo "| Notebook Processing | Skipped (merge mode) | - |"
  fi
  
  echo "| Documentation Build | $BUILD_RESULT | - |"
  echo "| Deprecation Management | $DEPRECATION_RESULT | - |"
  echo ""
} >> "$GITHUB_STEP_SUMMARY"

echo "✅ Job results table added"
cat "$GITHUB_STEP_SUMMARY"

## 4. Display Execution Details

Add execution details including processed notebooks and affected directories.

In [ ]:
#!/bin/bash

# Set example notebook data - NOTE: This is the problematic data from your error
# The issue was with malformed JSON - missing quotes around array
NOTEBOOKS='["notebooks/hello-universe/Classifying_JWST-HST_galaxy_mergers_with_CNNs/Classifying_JWST-HST_galaxy_mergers_with_CNNs.ipynb"]'
AFFECTED_DIRS='["notebooks/hello-universe/Classifying_JWST-HST_galaxy_mergers_with_CNNs"]'

# Add execution details header
{
  echo "### 📝 Execution Details"
} >> "$GITHUB_STEP_SUMMARY"

echo "✅ Execution details header added"
echo "Notebooks variable: $NOTEBOOKS"
echo "Affected directories: $AFFECTED_DIRS"

## 5. Determine Notebook Count (Robust Error Handling)

This is the critical section where your original script failed. Let's implement robust JSON parsing with proper error handling.

In [ ]:
#!/bin/bash

# Function to safely count notebooks with proper error handling
count_notebooks() {
    local notebooks_json="$1"
    local count=0
    
    # Check for null, empty, or invalid cases first
    if [ -z "$notebooks_json" ] || [ "$notebooks_json" = "null" ] || [ "$notebooks_json" = "[]" ]; then
        echo "0"
        return
    fi
    
    # Try to parse with jq, with fallback
    if command -v jq >/dev/null 2>&1; then
        # Use jq with error handling
        count=$(echo "$notebooks_json" | jq -r 'length' 2>/dev/null)
        
        # Check if jq parsing was successful
        if [ $? -eq 0 ] && [ "$count" != "null" ] && [ -n "$count" ]; then
            echo "$count"
        else
            echo "0"
        fi
    else
        # Fallback method without jq - count comma-separated items
        # Remove brackets and count items
        clean_json=$(echo "$notebooks_json" | sed 's/^\[//;s/\]$//')
        if [ -n "$clean_json" ]; then
            # Count commas and add 1, or 1 if no commas
            count=$(echo "$clean_json" | grep -o ',' | wc -l)
            count=$((count + 1))
            echo "$count"
        else
            echo "0"
        fi
    fi
}

# Test with various inputs
echo "Testing notebook counting:"
echo "Valid JSON: $(count_notebooks "$NOTEBOOKS")"
echo "Empty array: $(count_notebooks '[]')"
echo "Null: $(count_notebooks 'null')"
echo "Invalid JSON: $(count_notebooks '["malformed')"

# Set the actual notebook count
NOTEBOOK_COUNT=$(count_notebooks "$NOTEBOOKS")
echo "Final notebook count: $NOTEBOOK_COUNT"

## 6. Determine Overall Workflow Status

Count successful and failed jobs to determine the overall workflow status.

In [ ]:
#!/bin/bash

# Count successful and failed jobs
SUCCESS_COUNT=0
FAILURE_COUNT=0

# Check each job result
for result in "$SETUP_RESULT" "$PROCESS_RESULT" "$BUILD_RESULT" "$DEPRECATION_RESULT"; do
    case "$result" in
        "success") 
            SUCCESS_COUNT=$((SUCCESS_COUNT + 1)) 
            ;;
        "failure") 
            FAILURE_COUNT=$((FAILURE_COUNT + 1)) 
            ;;
        "skipped") 
            # Don't count skipped jobs as failures
            ;;
        "cancelled") 
            FAILURE_COUNT=$((FAILURE_COUNT + 1)) 
            ;;
    esac
done

echo "Job Status Summary:"
echo "- Success count: $SUCCESS_COUNT"
echo "- Failure count: $FAILURE_COUNT"
echo "- Setup: $SETUP_RESULT"
echo "- Process: $PROCESS_RESULT"
echo "- Build: $BUILD_RESULT" 
echo "- Deprecation: $DEPRECATION_RESULT"

## 7. Display Final Status Indicator

Based on the workflow status, append the appropriate final message to the summary.

In [ ]:
#!/bin/bash

# Add execution details with notebook count
{
  echo ""
  echo "**Processed Notebooks**: $NOTEBOOK_COUNT"
  echo ""
} >> "$GITHUB_STEP_SUMMARY"

# Special handling for process-notebooks in merge mode
if [ "$EXECUTION_MODE" = "merge" ] && [ "$PROCESS_RESULT" = "skipped" ]; then
    {
        echo "**Note**: Notebook processing was intentionally skipped in merge mode"
        echo ""
    } >> "$GITHUB_STEP_SUMMARY"
fi

# Add final status based on results
{
    if [ "$FAILURE_COUNT" -eq 0 ]; then
        echo "## 🎉 Workflow completed successfully!"
        echo "*All required jobs completed without errors*"
    elif [ "$SUCCESS_COUNT" -gt 0 ]; then
        echo "## ⚠️ Workflow completed with partial success"
        echo "*Some jobs succeeded but $FAILURE_COUNT job(s) failed - see error details above*"
    else
        echo "## ❌ Workflow failed"
        echo "*Multiple critical failures occurred - see error details above*"
    fi
} >> "$GITHUB_STEP_SUMMARY"

echo "✅ Final status added to summary"

## Complete Summary Output

Let's display the complete workflow summary that was generated:

In [ ]:
#!/bin/bash

echo "📄 Complete Workflow Summary:"
echo "================================"
cat "$GITHUB_STEP_SUMMARY"
echo "================================"
echo ""
echo "✅ Summary generation completed successfully!"
echo "   - Total lines: $(wc -l < "$GITHUB_STEP_SUMMARY")"
echo "   - File size: $(wc -c < "$GITHUB_STEP_SUMMARY") bytes"

## Troubleshooting Your Original Error

The error you encountered (`Process completed with exit code 5`) was likely caused by:

### Issue 1: Malformed JSON
Your original `NOTEBOOKS` variable had malformed JSON:
```bash
NOTEBOOKS="["notebooks/hello-universe/..."]"  # Missing quotes around the string
```

Should be:
```bash
NOTEBOOKS='["notebooks/hello-universe/..."]'  # Properly quoted JSON array
```

### Issue 2: Bash -e Flag
The `-e` flag causes bash to exit immediately on any command failure. When `jq` fails to parse invalid JSON, it returns a non-zero exit code, causing the script to terminate.

### Issue 3: No Error Handling
The original script didn't handle cases where:
- `jq` is not available
- JSON parsing fails
- Variables are null or empty

### Solutions Applied:
1. **Robust JSON validation** before parsing
2. **Fallback methods** when `jq` fails
3. **Proper error handling** with null checks
4. **Function-based approach** for reusability

### Fixed Script Structure:
```bash
# Use proper JSON quoting
NOTEBOOKS='["notebook1.ipynb", "notebook2.ipynb"]'

# Add error handling
if [ "$NOTEBOOKS" = "null" ] || [ "$NOTEBOOKS" = "" ] || [ "$NOTEBOOKS" = "[]" ]; then
    NOTEBOOK_COUNT=0
else
    NOTEBOOK_COUNT=$(echo "$NOTEBOOKS" | jq -r 'length' 2>/dev/null || echo "0")
fi
```